In [1]:
import pandas as pd
from utils import Options
import pickle as pkl
import numpy as np
from glob import glob
from readresults.readres_mvpa import merge_results, parse_roi_info, get_subj_avg
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [2]:
reslist = glob('../../MVPA_results/mainanalysis.csv')

In [3]:
results = merge_results(reslist)

In [4]:
results

,correct,output,rawdistance,distance,chunk,view,expected,subject,roi,approach,traindataformat,testdataformat,traintask,testtask,trainmodel,testmodel
0,0.666667,0.666667,-1.244251,0.483668,1.0,1,True,sub-001,ba-17-18_L_contr-objscrvsbas_top-100,traintest,betas,betas,train,test,5,15
1,1.000000,0.000000,1.695260,0.862563,1.0,1,True,sub-001,ba-17-18_L_contr-objscrvsbas_top-100,traintest,betas,betas,train,test,5,15
2,1.000000,1.000000,-2.582819,1.278971,2.0,1,True,sub-001,ba-17-18_L_contr-objscrvsbas_top-100,traintest,betas,betas,train,test,5,15
3,0.000000,1.000000,-0.567846,-0.266178,2.0,1,True,sub-001,ba-17-18_L_contr-objscrvsbas_top-100,traintest,betas,betas,train,test,5,15
4,1.000000,1.000000,-0.849631,0.380785,3.0,1,True,sub-001,ba-17-18_L_contr-objscrvsbas_top-100,traintest,betas,betas,train,test,5,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783019,0.000000,1.000000,-5.621240,-0.877734,3.0,2,NaN,sub-035,LO_R,traintest,betas,betas,test,train,17,5
783020,1.000000,1.000000,-0.699427,0.132047,3.0,2,NaN,sub-035,LO_R,traintest,betas,betas,test,train,17,5
783021,0.000000,1.000000,-4.216521,-0.664910,3.0,2,NaN,sub-035,LO_R,traintest,betas,betas,test,train,17,5
783022,1.000000,1.000000,-1.742733,0.290115,3.0,2,NaN,sub-035,LO_R,traintest,betas,betas,test,train,17,5


In [8]:
results.view.unique()

array([1, 2])

In [9]:
avgres = get_subj_avg(results, avg_decodedirs=False)

In [13]:
avgres.drop(['view'], axis=1)

,subject,roi,approach,traindataformat,testdataformat,traintask,testtask,trainmodel,testmodel,correct,output,rawdistance,distance
0,sub-001,LO_L,traintest,betas,betas,test,train,15,5,0.522222,0.494444,2.102088e-02,0.164589
1,sub-001,LO_L,traintest,betas,betas,test,train,17,5,0.616667,0.516667,-1.701254e-07,0.214297
2,sub-001,LO_L,traintest,betas,betas,train,test,5,15,0.547619,0.523810,-1.546702e-01,0.205074
3,sub-001,LO_L,traintest,betas,betas,train,test,5,17,0.571429,0.571429,-1.546699e-01,0.262475
4,sub-001,LO_L_contr-objvscr_top-100,traintest,betas,betas,test,train,15,5,0.558333,0.519444,-1.670378e-02,0.152644
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11859,sub-035,ba-17-18_R_contr-objscrvsbas_top-800,traintest,betas,betas,train,test,5,17,0.750000,0.464286,-1.846105e-01,0.571646
11860,sub-035,ba-17-18_R_contr-objscrvsbas_top-900,traintest,betas,betas,test,train,15,5,0.655556,0.483333,4.099376e-02,0.418003
11861,sub-035,ba-17-18_R_contr-objscrvsbas_top-900,traintest,betas,betas,test,train,17,5,0.783333,0.516667,-4.266198e-08,0.548363
11862,sub-035,ba-17-18_R_contr-objscrvsbas_top-900,traintest,betas,betas,train,test,5,15,0.678571,0.559524,-1.645248e-01,0.442570


In [ ]:
avgres = parse_roi_info(avgres)

In [ ]:
avgres.roi.unique()

In [ ]:
results[results['approach']=='CV'].chunk.unique()

In [ ]:
from readresults.readres_mvpa import parse_roi_info

In [ ]:
results_avg = results.groupby(['subject', 'roi']).mean().reset_index()

In [ ]:
results = parse_roi_info(results)

In [ ]:
results = results.drop(['chunk'], axis=1)

In [ ]:
ind_vars = ['subject', 'roi', 'approach', 
            'traindataformat', 'testdataformat', 'traintask',
            'testtask', 'trainmodel', 'testmodel', 
            'hemi', 'contrast', 'nvoxels']
ind_vars = [i for i in ind_vars if i in results.columns and results[i].nunique() > 0]

In [ ]:
results_avg = results.groupby(ind_vars).mean().reset_index()

In [ ]:
results_avg = results_avg.drop(['chunk'], axis=1)

In [ ]:
results_avg.head()

In [ ]:
sns.barplot(data=results_avg, x='roi', y='distance')

In [ ]:
allinfo = 'LO_contr-objscrvsbas_top-1000'.split('_')

In [ ]:
allinfo[1] in ['L', 'R']

In [ ]:
allinfo

In [ ]:
contrindx = 1

In [ ]:
allinfo[contrindx].split('contr-')[1]

In [ ]:
a = [None, None, None]
all(v is None for v in a)

In [ ]:
sns.barplot(data=results, x='roi', y='correct')

In [ ]:
results[results['roi'].isin(['rBA17_18_L', 'rBA17_18_R',
                             'ba-17-18_L', 'ba-17-18_R'])].distance.mean()

In [ ]:
outputs = [0.3, 4.0, 6.9, 2.0, 3.5, 0.1]
targets = [4.7, 3.2, 1.0, 2.9, 3.2, 6.8]
splits = [1, 1, 1, 2, 2, 2]
example_res = pd.DataFrame(
    list(zip(outputs, targets, splits)),
    columns=['output', 'target', 'split'])

In [ ]:
example_res

In [ ]:
x = np.array([3, 4, 5])
np.isin(x, [5])

In [ ]:
example_res = np.nan
isinstance(example_res, float) and pd.isnull(example_res)

In [ ]:
pd.isna(example_res).all()

In [ ]:
outs2 = [0.5, 0.65, 1.2]
splits2 = [3, 3, 3]
example_res_2 = pd.DataFrame(
    list(zip(outs2, splits2)),
    columns=['output', 'split'])

In [ ]:
all_res = pd.concat([example_res, example_res_2])

In [ ]:
all_res

In [ ]:
for s in sorted(example_res.split.unique()):
    thissplitlength = len(example_res[example_res['split']==s])
    example_res.loc[example_res['split']==s, 'sample'] = list(range(thissplitlength))

In [ ]:
groupedres = example_res.groupby('sample').mean().reset_index().drop(['sample', 'split'], axis=1)

In [ ]:
groupedres

In [ ]:
agg_functions = {'output': 'mean', 'target': 'sum'}

#create new DataFrame by combining rows with same id values
res_new = example_res.groupby(example_res['split']).aggregate(
    agg_functions)

In [ ]:
res_new

In [ ]:
with open('/project/3018040.05/MVPA_results/traintest.pkl', 'rb') as f:
    results = pkl.load(f)

In [ ]:
res = results['train-trainm3-test-testm3_ba-17-18_L_betas'][0]
accuracy = res[:, 0]==res[:, 1]
res_df = pd.DataFrame(res, 
                      columns = ['output', 'target', 'distance', 'chunk'])

In [ ]:
res_df['trialno'] = 1

In [ ]:
res_df.to_numpy()

In [ ]:
accuracy.shape

In [ ]:
random = np.random.rand(accuracy.shape[0])

In [ ]:
random

In [ ]:
output = (res[:, 0] == np.unique(res[:, 1])[0]).astype(int)

In [ ]:
output

In [ ]:
np.mean([True, True, False])

In [ ]:
outputs = 

In [ ]:
res_df.distance.dtype

In [ ]:
results['train-trainm3-test-testm3_ba-17-18_L_betas'][0].shape

In [ ]:
opt = Options()

In [ ]:
import configs

In [ ]:
dir(configs)

In [ ]:
configs.mvpa_outdir

In [ ]:
from configs import mvpa_outdir as data_dir

In [ ]:
from six.moves import cPickle as pickle

In [ ]:
data_dir

In [ ]:
mvpa_datadir

In [ ]:
x

In [ ]:
opt.sub = 'sub-001'

In [ ]:
opt.sub

In [ ]:
eventsfile = '/project/3018040.05/bids/sub-001/func/sub-001_task-test_run-1_events.tsv'
behavfile = '/project/3018040.05/bids/sub-001/func/sub-001_task-test_beh.tsv'
events = pd.read_csv(eventsfile, sep='\t')
behav = pd.read_csv(behavfile, sep='\t')

In [ ]:
events

In [ ]:
bedscenes = [2, 5, 6, 7, 9, 13, 14, 17, 18, 20]
bedindx = behav.index[behav['Scene'].isin(bedscenes)]
couchindx = behav.index[~behav['Scene'].isin(bedscenes)]

In [ ]:
bedindx

In [ ]:
couchindx